# Graph Dataset Loading and Batching
---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from os import path

import networkx as nx
from simgnn.datautils import *

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,10) # use larger for presentation
matplotlib.rcParams['font.size']= 14 # use 14 for presentation

In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
dtype = torch.float32
print(f'Deafults:\n |-device: {device}\n |-dtype : {dtype}')

Deafults:
 |-device: cuda
 |-dtype : torch.float32


## Dataset location

In [81]:
data_root = '../../../dataDIR/simgnn_data/raw'# location of all datasets
data_path = path.join(data_root,'17Nov2020')

In [82]:
!ls $data_path

graph_dict.pkl	simul_Ka.npy	     simul_Length.npy	  simul_t.npy
simul_A0.npy	simul_Kp.npy	     simul_P0.npy	  simul_vtxpos.npy
simul_Area.npy	simul_Lambda_ij.npy  simul_Perimeter.npy  t_Energy_maxSpeed.npy


Representation of cells in the monolayer graph:
- dict of lists with cell numbers $0, 1, \dots N$ as keys, and edges (values).
- edge ind-s **start from 1** in the cells (values), don't forget to **convert** when indexing edges tensor (e.g. `edge_ID=np.abs(ID)-1`). Negative indices indicate reversed order for vertices (cell is defined as a closed region formed by edges connected *end-to-end*). Use `np.sign(cells[c])` to find reversed edges for cell `c`.
    - e.g. get edges of cell `c` : `np.abs(mg_dict['cells'][c])-1` (`mg_dict` is monolayer graph dict loaded with `datautils.load_graph`)

In [83]:
# load dict of monolayer graph
mg_dict = load_graph(path.join(data_path,'graph_dict.pkl'))

# Load vertex positions from simulation results: Frames x Vertices x Positions
vx_pos = load_array(path.join(data_path,'simul_vtxpos.npy'))

# init-l vertex positions (1st frame)
vx0 = vx_pos[0]

# convert edges list to tensor
edges = torch.tensor(mg_dict['edges'],dtype=torch.long)

In [84]:
edge_lengths = load_array(path.join(data_path, 'simul_Length.npy'))
Lambda_ij = load_array(path.join(data_path,'simul_Lambda_ij.npy'))
# assert Lambda_ij.shape[1]!=edges.shape[0]

A0 = load_array(path.join(data_path, 'simul_A0.npy'))
Area = load_array(path.join(data_path, 'simul_Area.npy'))
Ka = load_array(path.join(data_path, 'simul_Ka.npy'))

P0 = load_array(path.join(data_path, 'simul_P0.npy'))
Perims = load_array(path.join(data_path, 'simul_Perimeter.npy'))
Kp = load_array(path.join(data_path, 'simul_Kp.npy'))

In [37]:
# edge_lengths.max()
# plt.hist(Perims.ravel(),bins=100);

In [38]:
# plt.hist(Area.ravel(),bins=100);

In [39]:
# plt.hist((edge_lengths.ravel()), bins=50, density=True);
# plt.xlabel('Edge Lengths (a.u.)')
# plt.ylabel('Frequency (normalised)')
# plt.title('10x10 cell monolayer trial with random oscillations\n(17-Nov-2020; 100 sample frames from a movie)')
# plt.savefig('edge_length_17Nov2020.png',dpi=100);

In [40]:
# import networkx as nx
# np.random.seed(42)

# cell_samples = np.random.choice(list(mg_dict['cells'].keys()),10)

# mg_nx = nx.Graph(mg_dict['edges'])
# nx.draw(mg_nx,pos=dict(zip(range(vx0.shape[0]),vx0)),
#         node_size=20,width=4,node_color='#FF00FF',edge_color='#51C5FF')
# for c in cell_samples:
#     c_vxs = vx0[edges[(np.abs(mg_dict['cells'][c])-1)].view(-1,),:]
#     c_loc = c_vxs.mean(axis=0)
#     plt.plot(c_loc[0],c_loc[1],'ro',ms=10)
#     for vx_i in c_vxs:
#         plt.plot([vx_i[0],c_loc[0]], [vx_i[1],c_loc[1]],lw=4)

## Working with PyTorch Geometric

`torch_geometric.data.Data` attributes:
- `.x` : node features `[num_nodes, num_node_features]`
- `.edge_index` : edges as \[source, target\] node index pairs (COO format) with shape `[2, num_edges]` and `torch.long` type (int).
- `.edge_attr`: edge features with shape `[num_edges, num_edge_features]`
- `.y` : target, arbitrary shape, if node-level target then `[num_nodes, *]`, or graph-level target `[1, *]`
- `.pos`: node positions `[num_nodes, num_dims]`.
- can add additional attributes, e.g. "cells"

In [41]:
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import torch_geometric.transforms as T

In [48]:
data = Data()
data.num_nodes = vx0.shape[0]
# undirected graph {ij, ji} pairs
data.edge_index =  torch.cat( [edges.T.contiguous(), edges.fliplr().T.contiguous()], axis=1)
data.pos = torch.from_numpy(vx0).type(dtype)

In [85]:
!ls $data_path

graph_dict.pkl	simul_Ka.npy	     simul_Length.npy	  simul_t.npy
simul_A0.npy	simul_Kp.npy	     simul_P0.npy	  simul_vtxpos.npy
simul_Area.npy	simul_Lambda_ij.npy  simul_Perimeter.npy  t_Energy_maxSpeed.npy


In [51]:
data.edge_index

tensor([[ 21,  22,   3,  ..., 235, 232, 238],
        [ 22,   3,   2,  ..., 210, 239, 233]])

- **Node features** : velocities from previous frames (~ 5 frames--> Alvaro Sanchez-Gonzalez, *et al.* 2020 \[ASG2020\])
- **Edge features** : can use edge directions (optional, might help to speed up training)
- **Current position** : technically a node feature, and needs to be normalized but not processed by the network.
- Positions (Cartesian) to polar edge attributes: `transforms.Polar` \[[link](https://pytorch-geometric.readthedocs.io/en/latest/modules/transforms.html#torch_geometric.transforms.Polar)\], I can also implement or use the `transforms.Cartesian` which computes direction vectors (position pairs -to- normalized direction vectors).
- need transform for velocity noise (use src from example transforms above and ASG2020 paper)
- Trail movie: max edge length ~ 3.25 a.u.

In [97]:
data.keys

['edge_index', 'pos']

In [89]:
# Compare computed and original *edge lengths*
# dist_fn = T.Distance(norm=False)
# compute cartesian distances
# dist_fn(data);

In [177]:
from torch_geometric.data import Dataset
from os import path, listdir

class VertexDynamics(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        '''
        Assumes `root` dir contains folder named `raw` with all vertex dynamics simulation results
        for tracing vertex trajectories, building graphs, and variables for computing edge tensions 
        and cell pressures.
        
        Arg-s:
        - root : path to a root directory that contains folder with raw dataset(s) in a folder named "raw".
        Raw datasets should be placed into separate folders each containing a separate simulation results.
        E.g. root contains ["raw", "processed", ...], and in folder "raw/" we should have ["simul1", "simul2", ...]
        - transform :  transform(s) for graph datasets (e.g. from torch_geometric.transforms )
        - pre_transform : transform(s) for data pre-processing (resulting graphs are saved in "preprocessed" folder)
        and used as this dataset's sample graphs.
        '''
        super(VertexDynamics, self).__init__(root, transform, pre_transform)
        
        self.raw_dir_path = path.join(self.root,'raw')
        assert path.isdir(self.raw_dir_path), f'Folder "{self.root}" does not contain folder named "raw".'

    @property
    def raw_file_names(self):
        raw_dirs = [folder_i for folder_i in listdir(self.raw_dir_path) 
                    if path.isdir( path.join( self.raw_dir_path, folder_i))]
        file_names = [path.join(dir_i, file_i) for dir_i in raw_dirs
                      for file_i in listdir(path.join(self.raw_dir_path,dir_i))] 
        return file_names

    @property
    def processed_file_names(self):
        #self.processed_dir # root/processed_dir
        return []#['data_1.pt', 'data_2.pt', ...]

    def process(self):
        pass
#         i = 0
#         for raw_path in self.raw_paths:
#             # Read data from `raw_path`.
#             data = Data(...)

#             if self.pre_filter is not None and not self.pre_filter(data):
#                 continue

#             if self.pre_transform is not None:
#                 data = self.pre_transform(data)

#             torch.save(data, osp.join(self.processed_dir, 'data_{}.pt'.format(i)))
#             i += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        pass
        #data = torch.load(osp.join(self.processed_dir, 'data_{}.pt'.format(idx)))
        return data

In [183]:
vtxdata = VertexDynamics('../../../dataDIR/simgnn_data')
vtxdata

VertexDynamics(0)

In [184]:
vtxdata.processed_dir

'../../../dataDIR/simgnn_data/processed'

## GCN tutorial

In [40]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print('#datasets:',len(dataset),'\n1st:',dataset[0],'\nName:',dataset)

# 2-layer GCN
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

print('Training 2-layer GCN.')
tot_epoch = 200
print_freq = tot_epoch//5
for epoch in range(tot_epoch):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if (epoch+1)%print_freq==0:
        model.eval()
        with torch.no_grad():
            _, pred = model(data).max(dim=1)
            correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
            print('Epoch {:5}| Accuracy: {:4.2f}% |'.format(epoch,100*correct / int(data.test_mask.sum())))

model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

#datasets: 1 
1st: Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708]) 
Name: Cora()
Training 2-layer GCN.
Epoch    39| Accuracy: 78.30% |
Epoch    79| Accuracy: 79.60% |
Epoch   119| Accuracy: 79.30% |
Epoch   159| Accuracy: 79.50% |
Epoch   199| Accuracy: 80.40% |
Accuracy: 0.8040
